In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [6]:
!ls 'drive/My Drive'

'Colab Notebooks'       MIVIA_DB4_dist	       Models	        Ravdess-Saved
 Extra		        MIVIA_out	       PAMAP2_Dataset   Video_Song
'Getting started.pdf'   Mivia_Processed_Data   Rakshak_Data     Video_Speech
 Misc		        Mivia-Saved	       Ravdess-Data


In [7]:
!pip install pyaudioanalysis pydub

     |████████████████████████████████| 51kB 2.4MB/s 


In [0]:
import numpy
from pydub import AudioSegment

import sys

if sys.version_info >= (3, 0):
    def xrange(*args, **kwargs):
        return iter(range(*args, **kwargs))

def readAudioFile(path):
    '''
    This function returns a numpy array that stores the audio samples of a specified WAV file
    '''
    try:
        audiofile = AudioSegment.from_file(path)

        if audiofile.sample_width == 2:
            data = numpy.fromstring(audiofile._data, numpy.int16)
        elif audiofile.sample_width == 4:
            data = numpy.fromstring(audiofile._data, numpy.int32)
        else:
            return (-1, -1)
        Fs = audiofile.frame_rate
        x = []
        for chn in xrange(audiofile.channels):
            x.append(data[chn::audiofile.channels])
        x = numpy.array(x).T

    except IOError: 
        print("Error: file not found or other I/O error.")
        return (-1,-1)

    if x.ndim == 2:
        if x.shape[1] == 1:
            x = x.flatten()

    return (Fs, x)

In [0]:
class Event:
    '''
          Support class to store info about the event
    '''

    def __init__(self, id, target, start, stop, background, data=None):
        self.id = id
        self.target = target
        self.startSecond = start
        self.stopSecond = stop
        self.background = background
        self.data = data

    def setData(self, data):
        self.data = data

    def getData(self):
        return self.data

    def getId(self):
        return self.id

    def getTarget(self):
        return self.target

    def getStartSecond(self):
        return self.startSecond

    def getStopSecond(self):
        return self.stopSecond

    def getBackground(self):
        return self.background

    def debug(self):
        print(self.getData())
        print(self.getId())
        print(self.getTarget())
        print(self.getStartSecond())
        print(self.getStopSecond())
        print(self.getBackground())

In [0]:
import xml.etree.ElementTree as et


class DatasetPreprocessing:

    def __init__(self):
        self.frameSize = 0.300
        self.overlap = 0.150
        self.tolerance = 0.2

    def parseXml(self, filename):
        tree = et.parse(filename)
        root = tree.getroot()

        background = ""
        for j in root.findall("./background/item"):
            background = background + "+" + j.find('SUBCLASS').text
        background = background[1:]

        eventsList = []
        for i in root.findall("./events/item"):
            className = i.find('CLASS_NAME').text
            if(className.startswith("glass")):
                target = "0"
            elif(className.startswith("gunshots")):
                target = "1"
            else:
                target = "2"
            start = i.find('STARTSECOND').text
            stop = i.find('ENDSECOND').text

            event = Event(className[:-4], target, start, stop, background)
            eventsList.append(event)

        return eventsList

    def segmentation(self, xml_filename, wav_filename):
        eventsList = self.parseXml(xml_filename)

        [self.Fs, x] = readAudioFile(wav_filename)

        segmentedFileList = []
        curPos = 0
        N = len(x)
        Win = int(self.Fs*self.frameSize)
        Step = int(self.Fs*self.overlap)
        index = 0

        while curPos + Win - 1 < N:
            curFrame = x[curPos:curPos + Win]

            curEvent = eventsList[index]
            start = float(curEvent.getStartSecond())*self.Fs
            stop = float(curEvent.getStopSecond())*self.Fs

            # debug print
            # print('start win ' + str(curPos))
            # print('start win +20% ' + str(curPos + self.tolerance*Win))
            # print('stop win ' + str(curPos + Win))
            # print('stop win -20% ' + str(curPos + Win - self.tolerance*Win))
            #
            # print('start event ' + str(start))
            # print('stop event ' + str(stop))

            if curPos + Win - self.tolerance*Win < start:
                target = "3"
                backgorundEvent = Event("other" + wav_filename[-12:-4], target, None, None,
                                              curEvent.getBackground(), curFrame)
                segmentedFileList.append(backgorundEvent)
            elif curPos + self.tolerance*Win >= start and curPos + Win - self.tolerance*Win <= stop:
                relevantEvent = Event(curEvent.getId(), curEvent.getTarget(),
                                            None, None,
                                            curEvent.getBackground(), curFrame)
                segmentedFileList.append(relevantEvent)
            elif curPos + self.tolerance*Win > stop:
                if index < len(eventsList)-1:
                    index += 1
                target = "3"
                backgorundEvent = Event("other" + wav_filename[-12:-4], target, None, None,
                                              curEvent.getBackground(), curFrame)
                segmentedFileList.append(backgorundEvent)

            curPos = curPos + Step

        return segmentedFileList

In [0]:

import numpy
from scipy.fftpack import fft
from scipy.fftpack.realtransforms import dct

eps = 0.00000001

""" Time-domain audio features """


def stZCR(frame):
    """Computes zero crossing rate of frame"""
    count = len(frame)
    countZ = numpy.sum(numpy.abs(numpy.diff(numpy.sign(frame)))) / 2
    return (numpy.float64(countZ) / numpy.float64(count-1.0))


def stEnergy(frame):
    """Computes signal energy of frame"""
    return numpy.sum(frame ** 2) / numpy.float64(len(frame))


def stEnergyEntropy(frame, numOfShortBlocks=10):
    """Computes entropy of energy"""
    Eol = numpy.sum(frame ** 2)    # total frame energy
    L = len(frame)
    subWinLength = int(numpy.floor(L / numOfShortBlocks))
    if L != subWinLength * numOfShortBlocks:
            frame = frame[0:subWinLength * numOfShortBlocks]
    # subWindows is of size [numOfShortBlocks x L]
    subWindows = frame.reshape(subWinLength, numOfShortBlocks, order='F').copy()

    # Compute normalized sub-frame energies:
    s = numpy.sum(subWindows ** 2, axis=0) / (Eol + eps)

    # Compute entropy of the normalized sub-frame energies:
    Entropy = -numpy.sum(s * numpy.log2(s + eps))
    return Entropy


""" Frequency-domain audio features """


def stSpectralCentroidAndSpread(X, fs):
    """Computes spectral centroid of frame (given abs(FFT))"""
    ind = (numpy.arange(1, len(X) + 1)) * (fs/(2.0 * len(X)))

    Xt = X.copy()
    Xt = Xt / Xt.max()
    NUM = numpy.sum(ind * Xt)
    DEN = numpy.sum(Xt) + eps

    # Centroid:
    C = (NUM / DEN)

    # Spread:
    S = numpy.sqrt(numpy.sum(((ind - C) ** 2) * Xt) / DEN)

    # Normalize:
    C = C / (fs / 2.0)
    S = S / (fs / 2.0)

    return (C, S)


def stSpectralEntropy(X, numOfShortBlocks=10):
    """Computes the spectral entropy"""
    L = len(X)                         # number of frame samples
    Eol = numpy.sum(X ** 2)            # total spectral energy

    subWinLength = int(numpy.floor(L / numOfShortBlocks))   # length of sub-frame
    if L != subWinLength * numOfShortBlocks:
        X = X[0:subWinLength * numOfShortBlocks]

    subWindows = X.reshape(subWinLength, numOfShortBlocks, order='F').copy()  # define sub-frames (using matrix reshape)
    s = numpy.sum(subWindows ** 2, axis=0) / (Eol + eps)                      # compute spectral sub-energies
    En = -numpy.sum(s*numpy.log2(s + eps))                                    # compute spectral entropy

    return En


def stSpectralFlux(X, Xprev):
    """
    Computes the spectral flux feature of the current frame
    ARGUMENTS:
        X:        the abs(fft) of the current frame
        Xpre:        the abs(fft) of the previous frame
    """
    # compute the spectral flux as the sum of square distances:
    sumX = numpy.sum(X + eps)
    sumPrevX = numpy.sum(Xprev + eps)
    F = numpy.sum((X / sumX - Xprev/sumPrevX) ** 2)

    return F


def stSpectralRollOff(X, c, fs):
    """Computes spectral roll-off"""
    totalEnergy = numpy.sum(X ** 2)
    fftLength = len(X)
    Thres = c*totalEnergy
    # Ffind the spectral rolloff as the frequency position where the respective spectral energy is equal to c*totalEnergy
    CumSum = numpy.cumsum(X ** 2) + eps
    [a, ] = numpy.nonzero(CumSum > Thres)
    if len(a) > 0:
        mC = numpy.float64(a[0]) / (float(fftLength))
    else:
        mC = 0.0
    return (mC)

def mfccInitFilterBanks(fs, nfft):
    """
    Computes the triangular filterbank for MFCC computation (used in the stFeatureExtraction function before the stMFCC function call)
    This function is taken from the scikits.talkbox library (MIT Licence):
    https://pypi.python.org/pypi/scikits.talkbox
    """

    # filter bank params:
    lowfreq = 133.33
    linsc = 200/3.
    logsc = 1.0711703
    numLinFiltTotal = 13
    numLogFilt = 27

    if fs < 8000:
        nlogfil = 5

    # Total number of filters
    nFiltTotal = numLinFiltTotal + numLogFilt

    # Compute frequency points of the triangle:
    freqs = numpy.zeros(nFiltTotal+2)
    freqs[:numLinFiltTotal] = lowfreq + numpy.arange(numLinFiltTotal) * linsc
    freqs[numLinFiltTotal:] = freqs[numLinFiltTotal-1] * logsc ** numpy.arange(1, numLogFilt + 3)
    heights = 2./(freqs[2:] - freqs[0:-2])

    # Compute filterbank coeff (in fft domain, in bins)
    fbank = numpy.zeros((nFiltTotal, nfft))
    nfreqs = numpy.arange(nfft) / (1. * nfft) * fs

    for i in range(nFiltTotal):
        lowTrFreq = freqs[i]
        cenTrFreq = freqs[i+1]
        highTrFreq = freqs[i+2]

        lid = numpy.arange(numpy.floor(lowTrFreq * nfft / fs) + 1, numpy.floor(cenTrFreq * nfft / fs) + 1, dtype=numpy.int)
        lslope = heights[i] / (cenTrFreq - lowTrFreq)
        rid = numpy.arange(numpy.floor(cenTrFreq * nfft / fs) + 1, numpy.floor(highTrFreq * nfft / fs) + 1, dtype=numpy.int)
        rslope = heights[i] / (highTrFreq - cenTrFreq)
        fbank[i][lid] = lslope * (nfreqs[lid] - lowTrFreq)
        fbank[i][rid] = rslope * (highTrFreq - nfreqs[rid])

    return fbank, freqs


def stMFCC(X, fbank, nceps):
    """
    Computes the MFCCs of a frame, given the fft mag
    ARGUMENTS:
        X:        fft magnitude abs(FFT)
        fbank:    filter bank (see mfccInitFilterBanks)
    RETURN
        ceps:     MFCCs (13 element vector)
    Note:    MFCC calculation is, in general, taken from the scikits.talkbox library (MIT Licence),
    #    with a small number of modifications to make it more compact and suitable for the pyAudioAnalysis Lib
    """

    mspec = numpy.log10(numpy.dot(X, fbank.T)+eps)
    ceps = dct(mspec, type=2, norm='ortho', axis=-1)[:nceps]
    return ceps

def stFeatureExtraction(signal, Fs, Win, Step):
    """
    This function implements the shor-term windowing process. For each short-term window a set of features is extracted.
    This results to a sequence of feature vectors, stored in a numpy matrix.
    ARGUMENTS
        signal:       the input signal samples
        Fs:           the sampling freq (in Hz)
        Win:          the short-term window size (in samples)
        Step:         the short-term window step (in samples)
    RETURNS
        stFeatures:   a numpy array (numOfFeatures x numOfShortTermWindows)
    """

    Win = int(Win)
    Step = int(Step)

    # Signal normalization
    signal = numpy.double(signal)

    signal = signal / (2.0 ** 15)
    DC = signal.mean()
    MAX = (numpy.abs(signal)).max()
    signal = (signal - DC) / (MAX + 0.0000000001)

    N = len(signal)                                # total number of samples
    curPos = 0
    countFrames = 0
    nFFT = int(Win / 2)

    [fbank, freqs] = mfccInitFilterBanks(Fs, nFFT)                # compute the triangular filter banks used in the mfcc calculation

    numOfTimeSpectralFeatures = 8
    numOfHarmonicFeatures = 0
    nceps = 13
    numOfChromaFeatures = 13
    totalNumOfFeatures = numOfTimeSpectralFeatures + nceps + numOfHarmonicFeatures + numOfChromaFeatures

    stFeatures = []
    while (curPos + Win - 1 < N):                        # for each short-term window until the end of signal
        countFrames += 1
        x = signal[curPos:curPos+Win]                    # get current window
        curPos = curPos + Step                           # update window position
        X = abs(fft(x))                                  # get fft magnitude
        X = X[0:nFFT]                                    # normalize fft
        X = X / len(X)
        if countFrames == 1:
            Xprev = X.copy()                             # keep previous fft mag (used in spectral flux)
        curFV = numpy.zeros((totalNumOfFeatures, 1))
        curFV[0] = stZCR(x)                              # zero crossing rate
        curFV[1] = stEnergy(x)                           # short-term energy
        curFV[2] = stEnergyEntropy(x)                    # short-term entropy of energy
        [curFV[3], curFV[4]] = stSpectralCentroidAndSpread(X, Fs)    # spectral centroid and spread
        curFV[5] = stSpectralEntropy(X)                  # spectral entropy
        curFV[6] = stSpectralFlux(X, Xprev)              # spectral flux
        curFV[7] = stSpectralRollOff(X, 0.90, Fs)        # spectral rolloff
        curFV[numOfTimeSpectralFeatures:numOfTimeSpectralFeatures+nceps, 0] = stMFCC(X, fbank, nceps).copy()    # MFCCs

        stFeatures.append(curFV)

        Xprev = X.copy()

    stFeatures = numpy.concatenate(stFeatures, 1)
    return stFeatures

In [9]:
import sys
import time

import numpy as np


class FeatureExtraction:
    '''
        This class implement features extraction using pyAudioAnalysis(a my extension to python3) on Mivia Audio Events Dataset.
        The data set:
        The MIVIA audio events data set is composed of a total of 6000 events for surveillance applications,
        namely glass breaking, gun shots and screams. The 6000 events are divided into a training set
        (composed of 4200 events) and a test set (composed of 1800 events).
        The data set is designed to provide each audio event at 6 different values of signal-to-noise ratio
        (namely 5dB, 10dB, 15dB, 20dB, 25dB and 30dB) and overimposed to different combinations of environmental
        sounds in order to simulate their occurrence in different ambiences.
        More details are reported on the web site http://mivia.unisa.it/datasets/audio-analysis/mivia-audio-events/.
        pyAudioAnalysis:
        pyAudioAnalysis is an open-source Python library that provides a wide range of audio analysis
        procedures including: feature extraction, classification of audio signals, supervised and unsupervised
        segmentation and content visualization. pyAudioAnalysis is licensed under the Apache License and is available
        at GitHub (https://github.com/tyiannak/pyAudioAnalysis/).
        For each frame the following features are computed:
            -Zero_Crossing_Rate_median, Zero_Crossing_Rate_median_absolute_deviation
            -Energy_median, Energy_median_absolute_deviation
            -Entropy_of_Energy_median, Entropy_of_Energy_median_absolute_deviation
            -Spectral_Centroid_median, Spectral_Centroid_median_absolute_deviation
            -Spectral_Spread_median, Spectral_Spread_median_absolute_deviation
            -Spectral_Entropy_median, Spectral_Entropy_median_absolute_deviation
            -Spectral_Flux_median, Spectral_Flux_median_absolute_deviation
            -Spectral_Rolloff_median, Spectral_Rolloff_median_absolute_deviation
            -MFCCs1_median, MFCCs1_median_absolute_deviation
            -MFCCs2_median, MFCCs2_median_absolute_deviation
            -MFCCs3_median, MFCCs3_median_absolute_deviation
            -MFCCs4_median, MFCCs4_median_absolute_deviation
            -MFCCs5_median, MFCCs5_median_absolute_deviation
            -MFCCs6_median, MFCCs6_median_absolute_deviation
            -MFCCs7_median, MFCCs7_median_absolute_deviation
            -MFCCs8_median, MFCCs8_median_absolute_deviation
            -MFCCs9_median, MFCCs9_median_absolute_deviation
            -MFCCs10_median, MFCCs10_median_absolute_deviation
            -MFCCs11_median, MFCCs11_median_absolute_deviation
            -MFCCs12_median, MFCCs12_median_absolute_deviation
            -MFCCs13_median, MFCCs13_median_absolute_deviation
        The method processTrainDataset() allows to process the training data of Mivia Audio Events Dataset.
        It parse the metadata within the xml files provided with the dataset in order to do the segmentation.
        In addition to the previuos ones, for each frame the following additinal "features" are computed:
            -target, [0 == glass, 1 == gunshots, 2 == screams, 3 ==  other]
            -frame, number of subframes
            -snr, snr in the range [1, 6]
            -id, id to easily recover the orginal file
            -background, background type [bells, crowd_claps, twistle, crowd, cars, household_app, rain, gaussian_noise]
                       |---------------------300ms--------------------|
                       |---------150ms---------|                        -------------> 1 sample, 42 features
                                   |---------150ms---------|
                                              |---------150ms---------|
    '''

    def __init__(self):
        self.dataPre = DatasetPreprocessing()
        self.segmentation = 0.300
        self.frameSize = 0.150
        self.frameStep = 0.075  # 50% overlap
        self.discard = 21  # number of original features to retain
        self.label = ['Zero_Crossing_Rate_median', 'Zero_Crossing_Rate_median_absolute_deviation',
                      'Energy_median', 'Energy_median_absolute_deviation',
                      'Entropy_of_Energy_median', 'Entropy_of_Energy_median_absolute_deviation',
                      'Spectral_Centroid_median', 'Spectral_Centroid_median_absolute_deviation',
                      'Spectral_Spread_median',  'Spectral_Spread_median_absolute_deviation',
                      'Spectral_Entropy_median', 'Spectral_Entropy_median_absolute_deviation',
                      'Spectral_Flux_median', 'Spectral_Flux_median_absolute_deviation',
                      'Spectral_Rolloff_median',  'Spectral_Rolloff_median_absolute_deviation',
                      'MFCCs1_median', 'MFCCs1_median_absolute_deviation',
                      'MFCCs2_median', 'MFCCs2_median_absolute_deviation',
                      'MFCCs3_median', 'MFCCs3_median_absolute_deviation',
                      'MFCCs4_median', 'MFCCs4_median_absolute_deviation',
                      'MFCCs5_median', 'MFCCs5_median_absolute_deviation',
                      'MFCCs6_median', 'MFCCs6_median_absolute_deviation',
                      'MFCCs7_median', 'MFCCs7_median_absolute_deviation',
                      'MFCCs8_median', 'MFCCs8_median_absolute_deviation',
                      'MFCCs9_median', 'MFCCs9_median_absolute_deviation',
                      'MFCCs10_median', 'MFCCs10_median_absolute_deviation',
                      'MFCCs11_median', 'MFCCs11_median_absolute_deviation',
                      'MFCCs12_median', 'MFCCs12_median_absolute_deviation',
                      'MFCCs13_median', 'MFCCs13_median_absolute_deviation',
                      'target', 'frame', 'snr', 'id', 'background']

    def extractFeatures(self, eventsList, Fs, snr):
        feature = []
        for event in eventsList:
            frame = event.getData()

            F = stFeatureExtraction(frame, Fs, self.frameSize * Fs, self.frameStep * Fs)
            raw_feature = F[:self.discard, :].T

            tmp = []
            for j in range(0, raw_feature.shape[1]):  # compute median and med for each columns
                feature_column = raw_feature[:, j]
                median = np.median(raw_feature[:, j])
                median_absolute_deviation = np.median(np.abs(feature_column - median))
                tmp.append(median)
                tmp.append(median_absolute_deviation)

            tmp.append(event.getTarget())  # add class label
            tmp.append(raw_feature.shape[0])  # add number of frame per signal
            tmp.append(snr)  # add snr
            tmp.append(event.getId())  # add id
            tmp.append(event.getBackground())  # add background type
            feature.append(tmp)

        return feature

    def update_progress(self, progress):
        barLength = 100  # Modify this to change the length of the progress bar
        status = ""
        if isinstance(progress, int):
            progress = float(progress)
        if not isinstance(progress, float):
            progress = 0
            status = "error: progress var must be float\r\n"
        if progress < 0:
            progress = 0
            status = "Halt...\r\n"
        if progress >= 1:
            progress = 1
            status = "Done...\r\n"
        block = int(round(barLength * progress))
        text = "\rPercent: [{0}] {1}% {2}".format("#" * block + "-" * (barLength - block), int(round(progress*100)), status)
        sys.stdout.write(text)
        sys.stdout.flush()

    def writeToCSV(self, feature, numRow, numCol, file):
        tmp = []
        for row in feature:
            tmp.append(np.append(numRow, row))
            numRow[0] += 1

        fmt = '%s,' * numCol
        fmt = fmt[:-1]

        np.savetxt(file, tmp, delimiter=",", fmt=fmt)

    def processDataset(self, path, wavNum, snrRange, outputFile):
        file = open(outputFile, 'ab')

        header = ","
        for x in self.label:
            header += x + ","

        file.write((header[:len(header)-1]+'\n').encode('ascii'))  # write .csv header

        total = (wavNum-1)*float(snrRange)
        cont = [0]
        for i in range(1, wavNum):  # process all wav
            if i < 10:
                xml_path = path + '0000' + str(i) + '.xml'
                wav_base_path = path + 'sounds/0000' + str(i)
            else:
                xml_path = path + '000' + str(i) + '.xml'
                wav_base_path = path + 'sounds/000' + str(i)

            for j in range(1, snrRange+1):  # process differents snr  1=30db ... 6=5db
                eventsList = self.dataPre.segmentation(xml_path, wav_base_path + '_' + str(j) + '.wav')
                feature = self.extractFeatures(eventsList, self.dataPre.Fs, snr=j)

                self.writeToCSV(feature, cont, len(self.label)+1, file)

                progress = float(((i - 1) * (snrRange) + j) / total)
                self.update_progress(progress)  # display progressbar

# #############################################################
# ############IN ORDER TO CREATE TRAINING DATASET##############
# #############################################################

# print("Pocessing training dataset...")
# print("")
# start_time = time.time()

# fe = FeatureExtraction()
# fe.processDataset(path='drive/My Drive/MIVIA_DB4_dist/training/',
#                        wavNum=67, snrRange=6,
#                        outputFile='drive/My Drive/MIVIA_out/validation3_training_dataset0300_overlap.csv')

# elapsed_time = time.time()-start_time
# print("")
# print("Elapsed time: " + str(elapsed_time))

# #############################################################
# #############################################################
# #############################################################


#############################################################
############IN ORDER TO CREATE TESTING DATASET##############
#############################################################

print("Pocessing testing dataset...")
print("")
start_time = time.time()

fe = FeatureExtraction()
fe.processDataset(path='drive/My Drive/MIVIA_DB4_dist/testing/',
                       wavNum=30, snrRange=6,
                       outputFile='drive/My Drive/MIVIA_out/validation3_testing_dataset0300_overlap.csv')

elapsed_time = time.time()-start_time
print("")
print("Elapsed time: " + str(elapsed_time))


#############################################################
#############################################################

Pocessing testing dataset...



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


Percent: [####################################################################################################] 100% Done...

Elapsed time: 1409.8333501815796


In [18]:
import pandas as pd
df = pd.read_csv('drive/My Drive/MIVIA_out/validation3_training_dataset0300_overlap.csv', index_col=0, skipfooter=1)
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  


,Zero_Crossing_Rate_median,Zero_Crossing_Rate_median_absolute_deviation,Energy_median,Energy_median_absolute_deviation,Entropy_of_Energy_median,Entropy_of_Energy_median_absolute_deviation,Spectral_Centroid_median,Spectral_Centroid_median_absolute_deviation,Spectral_Spread_median,Spectral_Spread_median_absolute_deviation,Spectral_Entropy_median,Spectral_Entropy_median_absolute_deviation,Spectral_Flux_median,Spectral_Flux_median_absolute_deviation,Spectral_Rolloff_median,Spectral_Rolloff_median_absolute_deviation,MFCCs1_median,MFCCs1_median_absolute_deviation,MFCCs2_median,MFCCs2_median_absolute_deviation,MFCCs3_median,MFCCs3_median_absolute_deviation,MFCCs4_median,MFCCs4_median_absolute_deviation,MFCCs5_median,MFCCs5_median_absolute_deviation,MFCCs6_median,MFCCs6_median_absolute_deviation,MFCCs7_median,MFCCs7_median_absolute_deviation,MFCCs8_median,MFCCs8_median_absolute_deviation,MFCCs9_median,MFCCs9_median_absolute_deviation,MFCCs10_median,MFCCs10_median_absolute_deviation,MFCCs11_median,MFCCs11_median_absolute_deviation,MFCCs12_median,MFCCs12_median_absolute_deviation,MFCCs13_median,MFCCs13_median_absolute_deviation,target,frame,snr,id,background
0,0.070640,0.001250,0.089931,0.004746,3.274921,0.002874,0.125528,0.003644,0.156826,0.002245,0.251838,0.001358,0.001318,0.000220,0.080417,0.000417,-19.906442,0.143701,1.103449,0.043328,-2.054377,0.055307,1.011898,0.066833,0.163481,0.009925,0.436943,0.048256,0.336700,0.030341,0.487895,0.003098,-0.279268,0.018899,-0.218039,0.000343,0.138110,0.020326,0.240087,0.065231,0.283265,0.006509,3,3,1,other/00001_1,cars
1,0.072307,0.001667,0.073158,0.005712,3.279981,0.005046,0.122908,0.001036,0.154578,0.000741,0.307084,0.035843,0.001243,0.000152,0.085417,0.001667,-19.887171,0.026316,1.060121,0.037143,-1.830998,0.077446,1.132301,0.091187,0.078376,0.075180,0.356909,0.107776,0.371649,0.065290,0.487895,0.006301,-0.150369,0.131108,-0.130923,0.084686,0.114187,0.019374,0.126117,0.005533,0.315269,0.013316,3,3,1,other/00001_1,cars
2,0.068973,0.001042,0.087305,0.002711,3.266151,0.024919,0.120518,0.005449,0.153251,0.010002,0.333204,0.009715,0.001315,0.000042,0.087917,0.000833,-19.769667,0.019302,0.930907,0.092074,-1.910269,0.029822,1.153760,0.002956,0.050596,0.023101,0.433022,0.031665,0.430932,0.015578,0.534810,0.005907,-0.019262,0.041047,-0.155399,0.109162,0.006313,0.058396,0.464995,0.004978,0.273656,0.041026,3,3,1,other/00001_1,cars
3,0.070640,0.000625,0.084670,0.002631,3.282819,0.000182,0.141394,0.001624,0.174841,0.002317,0.383753,0.007903,0.001097,0.000354,0.087917,0.001250,-19.887971,0.069843,0.812895,0.102074,-1.940091,0.000104,0.833571,0.022113,0.050596,0.068554,0.429411,0.003610,0.430932,0.042890,0.532882,0.001929,0.020674,0.095912,-0.268019,0.009767,0.209675,0.155497,0.464995,0.008228,0.331539,0.005330,3,3,1,other/00001_1,cars
4,0.069598,0.001042,0.089898,0.000920,3.282739,0.000603,0.129502,0.002766,0.166083,0.004425,0.383774,0.010090,0.001303,0.000205,0.087083,0.001667,-19.976660,0.069252,0.937053,0.002069,-1.961940,0.000652,0.990331,0.000174,0.119150,0.012493,0.615645,0.109478,0.426639,0.038598,0.374595,0.022381,0.078584,0.038003,-0.273015,0.004996,0.372149,0.006977,0.473223,0.034965,0.257556,0.066732,3,3,1,other/00001_1,cars


In [30]:
df_test = pd.read_csv('drive/My Drive/MIVIA_out/validation3_testing_dataset0300_overlap.csv', index_col=0, skipfooter=1)
df_test.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,Zero_Crossing_Rate_median,Zero_Crossing_Rate_median_absolute_deviation,Energy_median,Energy_median_absolute_deviation,Entropy_of_Energy_median,Entropy_of_Energy_median_absolute_deviation,Spectral_Centroid_median,Spectral_Centroid_median_absolute_deviation,Spectral_Spread_median,Spectral_Spread_median_absolute_deviation,Spectral_Entropy_median,Spectral_Entropy_median_absolute_deviation,Spectral_Flux_median,Spectral_Flux_median_absolute_deviation,Spectral_Rolloff_median,Spectral_Rolloff_median_absolute_deviation,MFCCs1_median,MFCCs1_median_absolute_deviation,MFCCs2_median,MFCCs2_median_absolute_deviation,MFCCs3_median,MFCCs3_median_absolute_deviation,MFCCs4_median,MFCCs4_median_absolute_deviation,MFCCs5_median,MFCCs5_median_absolute_deviation,MFCCs6_median,MFCCs6_median_absolute_deviation,MFCCs7_median,MFCCs7_median_absolute_deviation,MFCCs8_median,MFCCs8_median_absolute_deviation,MFCCs9_median,MFCCs9_median_absolute_deviation,MFCCs10_median,MFCCs10_median_absolute_deviation,MFCCs11_median,MFCCs11_median_absolute_deviation,MFCCs12_median,MFCCs12_median_absolute_deviation,MFCCs13_median,MFCCs13_median_absolute_deviation,target,frame,snr,id,background
0,0.062096,0.000208,0.081992,0.000237,3.257553,0.006397,0.124796,0.001577,0.168537,0.001550,0.419990,0.002014,0.000862,0.000145,0.081667,0.001250,-19.074372,0.045564,1.791450,0.018458,-0.861803,0.049865,0.398924,0.020454,-0.102819,0.011223,0.212847,0.099968,-0.089621,0.021333,0.349639,0.038426,0.062181,0.072146,0.241418,0.111849,0.251636,0.025943,0.313578,0.049119,0.004903,0.068618,3,3,1,other/00001_1,cars
1,0.058345,0.001667,0.074611,0.004201,3.274284,0.023015,0.125114,0.000281,0.168953,0.000404,0.442243,0.007510,0.001049,0.000033,0.086667,0.000417,-19.099951,0.025579,1.771272,0.028166,-0.923291,0.002203,0.457780,0.053899,-0.091596,0.021916,0.312815,0.005657,-0.079541,0.005420,0.328878,0.020760,-0.073718,0.033690,0.129569,0.066818,0.173599,0.022900,0.364741,0.025366,-0.008250,0.013153,3,3,1,other/00001_1,cars
2,0.064805,0.001042,0.073148,0.000643,3.262166,0.015391,0.131104,0.001989,0.174573,0.000466,0.447159,0.002587,0.000959,0.000186,0.087083,0.000000,-19.082894,0.019746,1.805914,0.097758,-0.787372,0.073777,0.511679,0.003817,-0.117061,0.012361,0.388970,0.049980,-0.074121,0.016436,0.301773,0.027106,0.044572,0.054209,0.196388,0.036610,0.307454,0.034697,0.240943,0.022059,-0.008250,0.024082,3,3,1,other/00001_1,cars
3,0.062721,0.002709,0.063335,0.003872,3.246875,0.049998,0.129104,0.000607,0.177334,0.002765,0.433628,0.010794,0.001156,0.000568,0.084583,0.001250,-19.293823,0.013087,1.914673,0.001312,-0.787370,0.006796,0.483988,0.115355,-0.165139,0.025958,0.357195,0.031774,-0.177044,0.051791,0.200912,0.001960,0.101221,0.002439,0.191706,0.031929,0.194134,0.008550,0.218883,0.053575,-0.020459,0.036144,3,3,1,other/00001_1,cars
4,0.055428,0.001667,0.074359,0.008826,3.217820,0.036978,0.130055,0.001560,0.177292,0.000091,0.330123,0.018632,0.001147,0.000261,0.079167,0.000000,-19.495586,0.015765,1.914670,0.049222,-0.714018,0.000067,0.647440,0.071688,0.027840,0.007404,0.357194,0.016259,-0.094455,0.030798,0.198622,0.002290,0.101221,0.038274,0.290148,0.046667,0.337406,0.108846,0.316524,0.007056,0.036530,0.017200,3,3,1,other/00001_1,cars


In [19]:
df.columns

Index(['Zero_Crossing_Rate_median',
       'Zero_Crossing_Rate_median_absolute_deviation', 'Energy_median',
       'Energy_median_absolute_deviation', 'Entropy_of_Energy_median',
       'Entropy_of_Energy_median_absolute_deviation',
       'Spectral_Centroid_median',
       'Spectral_Centroid_median_absolute_deviation', 'Spectral_Spread_median',
       'Spectral_Spread_median_absolute_deviation', 'Spectral_Entropy_median',
       'Spectral_Entropy_median_absolute_deviation', 'Spectral_Flux_median',
       'Spectral_Flux_median_absolute_deviation', 'Spectral_Rolloff_median',
       'Spectral_Rolloff_median_absolute_deviation', 'MFCCs1_median',
       'MFCCs1_median_absolute_deviation', 'MFCCs2_median',
       'MFCCs2_median_absolute_deviation', 'MFCCs3_median',
       'MFCCs3_median_absolute_deviation', 'MFCCs4_median',
       'MFCCs4_median_absolute_deviation', 'MFCCs5_median',
       'MFCCs5_median_absolute_deviation', 'MFCCs6_median',
       'MFCCs6_median_absolute_deviation', 'MFCCs

In [20]:
features_name = ['Zero_Crossing_Rate_median',
       'Zero_Crossing_Rate_median_absolute_deviation', 'Energy_median',
       'Energy_median_absolute_deviation', 'Entropy_of_Energy_median',
       'Entropy_of_Energy_median_absolute_deviation',
       'Spectral_Centroid_median',
       'Spectral_Centroid_median_absolute_deviation', 'Spectral_Spread_median',
       'Spectral_Spread_median_absolute_deviation', 'Spectral_Entropy_median',
       'Spectral_Entropy_median_absolute_deviation', 'Spectral_Flux_median',
       'Spectral_Flux_median_absolute_deviation', 'Spectral_Rolloff_median',
       'Spectral_Rolloff_median_absolute_deviation', 'MFCCs1_median',
       'MFCCs1_median_absolute_deviation', 'MFCCs2_median',
       'MFCCs2_median_absolute_deviation', 'MFCCs3_median',
       'MFCCs3_median_absolute_deviation', 'MFCCs4_median',
       'MFCCs4_median_absolute_deviation', 'MFCCs5_median',
       'MFCCs5_median_absolute_deviation', 'MFCCs6_median',
       'MFCCs6_median_absolute_deviation', 'MFCCs7_median',
       'MFCCs7_median_absolute_deviation', 'MFCCs8_median',
       'MFCCs8_median_absolute_deviation', 'MFCCs9_median',
       'MFCCs9_median_absolute_deviation', 'MFCCs10_median',
       'MFCCs10_median_absolute_deviation', 'MFCCs11_median',
       'MFCCs11_median_absolute_deviation', 'MFCCs12_median',
       'MFCCs12_median_absolute_deviation', 'MFCCs13_median',
       'MFCCs13_median_absolute_deviation']
len(features_name)

42

In [21]:
X_test = df_test.iloc[:,:42].values
X_test.shape

(22912, 42)

In [22]:
Y_test = df_test.iloc[:,42].values
Y_test.shape

(22912,)

In [31]:
X = df.iloc[:, :42].values
Y = df.iloc[:, 42].values
print(X.shape, Y.shape)

(191963, 42) (191963,)


In [1]:
!pip install tensorflow-gpu keras

     |████████████████████████████████| 377.0MB 51kB/s 
     |████████████████████████████████| 3.2MB 39.3MB/s 
ERROR: tensorflow 1.14.0rc1 has requirement tensorboard<1.14.0,>=1.13.0, but you'll have tensorboard 1.14.0 which is incompatible.
  Found existing installation: tensorboard 1.13.1
    Uninstalling tensorboard-1.13.1:
      Successfully uninstalled tensorboard-1.13.1


In [2]:
import tensorflow as tf
tf.__version__

'1.14.0'

In [3]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [27]:
import numpy as np
x_test = np.expand_dims(X_test, axis=2)
x_test.shape

(22912, 42, 1)

In [32]:
x_train = np.expand_dims(X, axis=2)
x_train.shape

(191963, 42, 1)

In [0]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint

model = Sequential()

model.add(Conv1D(128, 5,padding='same',
                 input_shape=(42,1)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(8))
model.add(Activation('softmax'))
opt = keras.optimizers.rmsprop(lr=0.00005, rho=0.9, epsilon=None, decay=0.0)

In [0]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [50]:
cnn = model.fit(x_train, Y, batch_size=16, epochs=10, validation_data=(x_test, Y_test))

Train on 191963 samples, validate on 22912 samples
Epoch 1/10
191963/191963 [==============================] - 47s 247us/step - loss: 0.3378 - acc: 0.8840 - val_loss: 0.3782 - val_acc: 0.8434
Epoch 2/10
191963/191963 [==============================] - 47s 247us/step - loss: 0.2155 - acc: 0.9248 - val_loss: 0.3076 - val_acc: 0.8955
Epoch 3/10
191963/191963 [==============================] - 46s 242us/step - loss: 0.1857 - acc: 0.9384 - val_loss: 0.2679 - val_acc: 0.9285
Epoch 4/10
191963/191963 [==============================] - 46s 240us/step - loss: 0.1665 - acc: 0.9461 - val_loss: 0.2323 - val_acc: 0.9426
Epoch 5/10
191963/191963 [==============================] - 47s 243us/step - loss: 0.1538 - acc: 0.9513 - val_loss: 0.3729 - val_acc: 0.8562
Epoch 6/10
191963/191963 [==============================] - 46s 239us/step - loss: 0.1431 - acc: 0.9559 - val_loss: 0.3204 - val_acc: 0.8828
Epoch 7/10
191963/191963 [==============================] - 46s 240us/step - loss: 0.1364 - acc: 0.9587

In [0]:
model.save('drive/My Drive/MIVIA_out/model.h5')